## Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Load in your data from kaggle.  
By working in a kaggle kernel, you can access the data directly from the competition, as well as make your submission without downloading your output file

In [ ]:
train = pd.read_csv('../input/edsa-climate-change-belief-analysis-2021/train.csv')
test = pd.read_csv('../input/edsa-climate-change-belief-analysis-2021/test.csv')
sample = pd.read_csv('../input/edsa-climate-change-belief-analysis-2021/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample.head()

In [ ]:
train.sentiment.value_counts()

In [ ]:
plot =train['sentiment'].value_counts()
sns.barplot(plot.index,plot)

## Splitting out the X variable from the target

In [ ]:
y = train['sentiment']
X = train['message']

## Turning text into something your model can read

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
X_vectorized = vectorizer.fit_transform(X)


## Splitting the training data into a training and validation set

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=.05,shuffle=True, stratify=y, random_state=42)

## Training the model and evaluating using the validation set 

In [ ]:
rfc = RandomForestClassifier(n_estimators=400)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_val)

## Checking the performance of our model on the validation set

In [ ]:
f1_score(y_val, rfc_pred, average="macro")

## Getting our test set ready 

In [ ]:
testx = test['message']
test_vect = vectorizer.transform(testx)

## Making predictions on the test set and adding a sentiment column to our original test df

In [ ]:
y_pred = rfc.predict(test_vect)

In [ ]:
test['sentiment'] = y_pred

In [ ]:
test.head()

## Creating an output csv for submission

In [ ]:
test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False)